In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm.contrib import tzip
import os

from utils import *

In [26]:
def transform_dataset(dataset, num_transforms=2, task=None, tran=None):
    df = init_transforms(task=task, tran=tran, meta=True)
    text, label = dataset['text'], dataset['label'] 
    new_text, new_label, trans = [], [], []
    if tran == 'SIB-mix':
        if type(text) == list:
            text = np.array(text, dtype=np.string_)
            label = np.array(label)
        batch_size= 1000
        for i in range(0, len(label), batch_size):
            text_batch = text[i:i+batch_size]
            label_batch = label[i:i+batch_size]
            batch = (text_batch, label_batch)
            t_trans = []
            n = 0
            while n < num_transforms:
                t_df   = df.sample(1)
                t_fn   = t_df['tran_fn'].iloc[0]
                t_name = t_df['transformation'].iloc[0]
                if t_name in trans:
                    continue
                else:
                    t_trans.append(t_name)
                batch, meta = t_fn(batch)
                if meta['change']:
                    n += 1
                else:
                    t_trans.remove(t_name)
            new_text.extend(batch[0].tolist())
            new_label.extend(batch[1].tolist())
            trans.append(t_trans)
    else:
        for X, y in tzip(text, label):
            t_trans = []
            n = 0
            while n < num_transforms:
                t_df   = df.sample(1)
                t_fn   = t_df['tran_fn'].iloc[0]
                t_name = t_df['transformation'].iloc[0]
                if t_name in t_trans:
                    continue
                else:
                    t_trans.append(t_name)
                X, y, meta = t_fn.transform_Xy(str(X), y)
                if meta['change']:
                    n += 1
                else:
                    t_trans.remove(t_name)
            new_text.append(X)
            new_label.append(y)
            trans.append(t_trans)
    return new_text, new_label, trans

In [4]:
df = init_transforms(task=None, tran=None, meta=True)

In [5]:
datasets = [('glue', 'sst2'), 'ag_news']
tasks = df.task_name.unique()
trans = df.tran_type.unique()

In [ ]:
n = 2 
for d in datasets:
    if type(d) == tuple and d[0] == 'glue':
        train, test = load_dataset(d[0], d[1], split=['train', 'test'])
        train.rename_column_('sentence', 'text')
        d = d[1]
    else:
        train, test = load_dataset(d, split=['train', 'test'])
    for task in tasks:
        task_df = df['task_name'] == task
        trans = df[task_df].tran_type.unique()
        for tran in trans:
            if d == 'ag_news' and task == 'sentiment':
                continue
            if d == 'sst2' and (task == 'topic' or tran == 'SIB-mix'):
                continue
            print(task, tran)
            if tran == 'SIB-mix':
                n = 1
            new_text, new_label, trans = transform_dataset(train, num_transforms=n, task=task, tran=tran)
            save_dir = os.path.join('assets', d, task, tran)
            npy_save(os.path.join(save_dir, 'text'), new_text)
            npy_save(os.path.join(save_dir, 'label'), new_label)
            npy_save(os.path.join(save_dir, 'trans'), trans)